In [1]:
# =========================
# 1. 配置区(只改这里)
# =========================

import os
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')

# 栅格路径
DAY_TIFS = [
    r"H:\Landsat_NL_Mector_90m_zscore\Landsat_RED_2020_90m_zscore.tif",
    r"H:\Landsat_NL_Mector_90m_zscore\Landsat_GREEN_2020_90m_zscore.tif",
    r"H:\Landsat_NL_Mector_90m_zscore\Landsat_BLUE_2020_90m_zscore.tif",
    r"H:\Landsat_NL_Mector_90m_zscore\Landsat_NIR_2020_90m_zscore.tif",
    r"H:\Landsat_NL_Mector_90m_zscore\Landsat_SWIR1_2020_90m_zscore.tif",
    r"H:\Landsat_NL_Mector_90m_zscore\Landsat_SWIR2_2020_90m_zscore.tif",
    r"H:\Landsat_NL_Mector_90m_zscore\Landsat_TEMP1_2020_90m_zscore.tif",
]
NIGHT_TIF = r"H:\Landsat_NL_Mector_90m_zscore\VIIRS_2020_90m_zscore.tif"

# 样本矢量
LABEL_SHP = r"H:\sample_2020\Sample_2020.shp"

# 四个组成比例字段
TARGET_FIELDS = ["F", "F_NF", "NF_F", "NF"]

# 输出目录
OUT_DIR = r"H:\model_outputs_2020_deep"
os.makedirs(OUT_DIR, exist_ok=True)

# 训练超参
PATCH_SIZE = 64
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5  # L2正则

# 类似论文的几种数据比例
FRACTIONS = [0.05, 0.10, 0.25, 0.50, 1.00]

# 要跑哪种模型
RUN_DAY = False
RUN_NIGHT = False
RUN_FUSION = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


Using device: cuda
GPU: Quadro P2200
Memory: 5.37 GB


In [2]:
# =========================
# 2. 数据集(改进版)
# =========================
import rasterio
import geopandas as gpd
from torch.utils.data import Dataset, DataLoader

class PatchPointDataset(Dataset):
    """
    改进点:
    1. 预筛有效样本
    2. 支持数据增强
    3. 确保标签是一个4维组成并归一化
    """
    def __init__(self, shp_path, day_paths, night_path, target_fields, 
                 patch=64, augment=False):
        self.gdf = gpd.read_file(shp_path).reset_index(drop=True)
        self.day_srcs = [rasterio.open(p) for p in day_paths]
        self.night_src = rasterio.open(night_path)
        self.target_fields = target_fields
        self.patch = patch
        self.augment = augment
        
        self.valid_idx = []
        hpatch = self.patch // 2
        first_src = self.day_srcs[0]

        print("⏳ 正在检查哪些样本在影像上有值...")
        for idx, row in self.gdf.iterrows():
            try:
                geom = row.geometry
                if geom is None or geom.is_empty:
                    continue
                if geom.geom_type != "Point":
                    geom = geom.centroid
                x, y = geom.x, geom.y
                r, c = rasterio.transform.rowcol(first_src.transform, x, y)
                
                # 边界检查
                if r < hpatch or c < hpatch or \
                   r >= first_src.height - hpatch or c >= first_src.width - hpatch:
                    continue
                
                window = rasterio.windows.Window(c - hpatch, r - hpatch, 
                                                self.patch, self.patch)

                ok = True
                for src in self.day_srcs:
                    arr = src.read(1, window=window, boundless=True, masked=True)
                    if arr.mask.all() or np.isnan(arr.filled(0)).all():
                        ok = False
                        break
                
                if ok:
                    narr = self.night_src.read(1, window=window, boundless=True, masked=True)
                    if narr.mask.all() or np.isnan(narr.filled(0)).all():
                        ok = False
                
                # 标签检查
                if ok:
                    vals = [row[f] for f in self.target_fields]
                    if any(v is None or (isinstance(v, float) and np.isnan(v)) for v in vals):
                        ok = False
                
                if ok:
                    self.valid_idx.append(idx)
                    
            except Exception as e:
                print(f"Warning: Sample {idx} failed: {e}")
                continue

        print(f"✅ 可用样本数: {len(self.valid_idx)} / {len(self.gdf)}")
        if len(self.valid_idx) == 0:
            raise RuntimeError("没有可用样本,请检查矢量和栅格是否对齐。")

    def __len__(self):
        return len(self.valid_idx)

    def __getitem__(self, i):
        real_idx = self.valid_idx[i]
        row = self.gdf.iloc[real_idx]
        geom = row.geometry
        if geom.geom_type != "Point":
            geom = geom.centroid
        x, y = geom.x, geom.y

        r, c = rasterio.transform.rowcol(self.day_srcs[0].transform, x, y)
        half = self.patch // 2
        window = rasterio.windows.Window(c - half, r - half, self.patch, self.patch)

        # 日间栈
        day_stack = []
        for src in self.day_srcs:
            arr = src.read(1, window=window, boundless=True, masked=True)
            day_stack.append(arr.filled(0).astype(np.float32))
        day = np.stack(day_stack, axis=0)

        # 夜间
        night_arr = self.night_src.read(1, window=window, boundless=True, masked=True)
        night = np.expand_dims(night_arr.filled(0).astype(np.float32), axis=0)

        # 数据增强
        if self.augment:
            if np.random.rand() > 0.5:
                day = np.flip(day, axis=1).copy()
                night = np.flip(night, axis=1).copy()
            if np.random.rand() > 0.5:
                day = np.flip(day, axis=2).copy()
                night = np.flip(night, axis=2).copy()
            k = np.random.randint(0, 4)
            if k > 0:
                day = np.rot90(day, k, axes=(1, 2)).copy()
                night = np.rot90(night, k, axes=(1, 2)).copy()

        # 标签
        vals = []
        for f in self.target_fields:
            v = row[f]
            v = 0.0 if (v is None or (isinstance(v, float) and np.isnan(v))) else v
            vals.append(float(v))
        y = np.array(vals, dtype=np.float32)
        
        # 归一化成组成
        s = y.sum()
        if s > 1e-6:
            y = y / s
        else:
            y = np.array([1.0] + [0.0]*(len(self.target_fields)-1), dtype=np.float32)

        return {
            "day": torch.from_numpy(day),
            "night": torch.from_numpy(night),
            "y": torch.from_numpy(y),
        }

    def close(self):
        for s in self.day_srcs:
            s.close()
        self.night_src.close()


In [3]:
# =========================
# 3. 模型定义(改进版)
# =========================
import torch.nn as nn
import torch.nn.functional as F

class SourceAdapter(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = out + identity
        out = self.relu(out)
        return out

class ConvStem(nn.Module):
    def __init__(self, in_channels, base_channels=64, num_blocks=2):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, base_channels, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(base_channels)
        self.relu = nn.ReLU(inplace=True)
        
        self.blocks = nn.Sequential(
            *[ResidualBlock(base_channels) for _ in range(num_blocks)]
        )
        
        self.conv2 = nn.Conv2d(base_channels, base_channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(base_channels)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.blocks(x)
        x = self.relu(self.bn2(self.conv2(x)))
        return x

class CompositionHead(nn.Module):
    def __init__(self, in_channels, n_comp=4):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, 3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout2d(0.1)
        self.conv2 = nn.Conv2d(in_channels, n_comp, 1)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)
        alpha_raw = self.conv2(x)
        alpha = F.softplus(alpha_raw) + 1.0  # 保证>0
        return alpha

def dirichlet_nll(y_true, alpha, eps=1e-7):
    """
    y_true: (B, C) or (B, C, 1, 1)
    alpha : (B, C, H, W)
    我们只取中心像元来做loss
    """
    if y_true.dim() == 2:
        y_true = y_true.unsqueeze(-1).unsqueeze(-1)  # (B,C,1,1)
    if alpha.dim() == 4:
        B, C, H, W = alpha.shape
        c = H // 2
        alpha = alpha[:, :, c, c]  # (B, C)

    alpha0 = alpha.sum(dim=1, keepdim=True)  # (B,1)
    logC = torch.lgamma(alpha0) - torch.lgamma(alpha).sum(dim=1, keepdim=True)

    y_safe = torch.clamp(y_true.squeeze(-1).squeeze(-1), min=eps, max=1.0-eps)
    logL = logC + ((alpha - 1) * torch.log(y_safe)).sum(dim=1, keepdim=True)
    return -logL.mean()

class DayOnlyNet(nn.Module):
    def __init__(self, day_channels=7, n_comp=4, shared_c=64):
        super().__init__()
        self.day_adapter = SourceAdapter(day_channels, shared_c)
        self.backbone = ConvStem(in_channels=shared_c, base_channels=shared_c)
        self.head = CompositionHead(in_channels=shared_c, n_comp=n_comp)
    
    def forward(self, day, night=None):
        d = self.day_adapter(day)
        x = self.backbone(d)
        alpha = self.head(x)
        return alpha

class NightOnlyNet(nn.Module):
    def __init__(self, night_channels=1, n_comp=4, shared_c=64):
        super().__init__()
        self.night_adapter = SourceAdapter(night_channels, shared_c)
        self.backbone = ConvStem(in_channels=shared_c, base_channels=shared_c)
        self.head = CompositionHead(in_channels=shared_c, n_comp=n_comp)
    
    def forward(self, day=None, night=None):
        n = self.night_adapter(night)
        x = self.backbone(n)
        alpha = self.head(x)
        return alpha

class FusionNet(nn.Module):
    def __init__(self, day_channels=7, night_channels=1, n_comp=4, shared_c=64):
        super().__init__()
        self.day_adapter = SourceAdapter(day_channels, shared_c)
        self.night_adapter = SourceAdapter(night_channels, shared_c)
        self.shared_backbone = ConvStem(in_channels=2*shared_c, base_channels=shared_c, num_blocks=3)
        self.head = CompositionHead(in_channels=shared_c, n_comp=n_comp)
    
    def forward(self, day, night):
        d = self.day_adapter(day)
        n = self.night_adapter(night)
        x = torch.cat([d, n], dim=1)
        x = self.shared_backbone(x)
        alpha = self.head(x)
        return alpha


In [4]:
# =========================
# 4. 单折 + 单一数据比例训练(改进版)
# =========================
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

plt.rcParams["font.size"] = 12
plt.rcParams["font.weight"] = "normal"

def train_one_fold_one_fraction(full_ds,
                                train_idx_full,
                                val_idx,
                                test_idx,
                                model_kind="fusion",
                                data_fraction=1.0,
                                fold_id=1):
    """
    单折单比例训练，并返回一个 metrics dict
    """
    # 1) 按比例裁训练集
    n_train_use = max(1, int(len(train_idx_full) * data_fraction))
    train_idx = train_idx_full[:n_train_use]

    train_subset = torch.utils.data.Subset(full_ds, list(train_idx))
    val_subset = torch.utils.data.Subset(full_ds, list(val_idx))
    test_subset = torch.utils.data.Subset(full_ds, list(test_idx))

    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, 
                            shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, 
                          shuffle=False, num_workers=0, pin_memory=True)
    test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, 
                           shuffle=False, num_workers=0, pin_memory=True)

    # 2) 选模型
    if model_kind == "day":
        model = DayOnlyNet(day_channels=len(DAY_TIFS), n_comp=len(TARGET_FIELDS)).to(DEVICE)
    elif model_kind == "night":
        model = NightOnlyNet(n_comp=len(TARGET_FIELDS)).to(DEVICE)
    else:
        model = FusionNet(day_channels=len(DAY_TIFS), night_channels=1, 
                         n_comp=len(TARGET_FIELDS)).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, 
                                  weight_decay=WEIGHT_DECAY)

    # ★★ 这里去掉 verbose 参数 ★★
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
    )

    best_val = 1e9
    best_state = None
    patience = 10
    no_improve = 0

    for epoch in range(EPOCHS):
        # ----- 训练 -----
        model.train()
        train_loss = 0.0
        for batch in train_loader:
            day = batch["day"].to(DEVICE, non_blocking=True)
            night = batch["night"].to(DEVICE, non_blocking=True)
            y = batch["y"].to(DEVICE, non_blocking=True)

            optimizer.zero_grad()
            if model_kind == "day":
                alpha = model(day, None)
            elif model_kind == "night":
                alpha = model(None, night)
            else:
                alpha = model(day, night)

            loss = dirichlet_nll(y, alpha)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item() * day.size(0)

        train_loss /= len(train_loader.dataset)

        # ----- 验证 -----
        model.eval()
        val_loss = 0.0
        y_true_all = []
        y_pred_all = []
        
        with torch.no_grad():
            for batch in val_loader:
                day = batch["day"].to(DEVICE, non_blocking=True)
                night = batch["night"].to(DEVICE, non_blocking=True)
                y = batch["y"].to(DEVICE, non_blocking=True)

                if model_kind == "day":
                    alpha = model(day, None)
                elif model_kind == "night":
                    alpha = model(None, night)
                else:
                    alpha = model(day, night)

                loss = dirichlet_nll(y, alpha)
                val_loss += loss.item() * day.size(0)

                H = alpha.shape[2]
                c = H // 2
                alpha_c = alpha[:, :, c, c]
                
                alpha_np = alpha_c.cpu().numpy()
                y_np = y.cpu().numpy()
                pred_np = alpha_np / np.clip(alpha_np.sum(axis=1, keepdims=True), 1e-6, None)
                
                y_true_all.append(y_np)
                y_pred_all.append(pred_np)

        val_loss /= len(val_loader.dataset)
        y_true_all = np.vstack(y_true_all)
        y_pred_all = np.vstack(y_pred_all)

        valid_mask = np.isfinite(y_true_all).all(axis=1) & np.isfinite(y_pred_all).all(axis=1)
        y_true_all = y_true_all[valid_mask]
        y_pred_all = y_pred_all[valid_mask]

        if y_true_all.shape[0] < 2:
            r2_mean = float("nan")
        else:
            y_true_all = np.clip(y_true_all, 0.0, 1.0)
            y_pred_all = np.clip(y_pred_all, 0.0, 1.0)
            r2_each = [r2_score(y_true_all[:, i], y_pred_all[:, i]) 
                      for i in range(len(TARGET_FIELDS))]
            r2_mean = float(np.mean(r2_each))

        if epoch % 10 == 0 or epoch == EPOCHS - 1:
            print(f"[Fold {fold_id}][frac {data_fraction:.2f}] "
                  f"epoch {epoch+1}/{EPOCHS} "
                  f"train_loss={train_loss:.4f} val_loss={val_loss:.4f} meanR²={r2_mean:.3f}")

        scheduler.step(val_loss)

        # 早停
        if val_loss < best_val - 1e-5:
            best_val = val_loss
            best_state = model.state_dict()
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # ---- 测试集评估 ----
    if best_state is not None:
        model.load_state_dict(best_state)
    
    model.eval()
    y_true_all = []
    y_pred_all = []
    
    with torch.no_grad():
        for batch in test_loader:
            day = batch["day"].to(DEVICE, non_blocking=True)
            night = batch["night"].to(DEVICE, non_blocking=True)
            y = batch["y"].to(DEVICE, non_blocking=True)

            if model_kind == "day":
                alpha = model(day, None)
            elif model_kind == "night":
                alpha = model(None, night)
            else:
                alpha = model(day, night)

            H = alpha.shape[2]
            c = H // 2
            alpha_c = alpha[:, :, c, c]

            alpha_np = alpha_c.cpu().numpy()
            y_np = y.cpu().numpy()
            pred_np = alpha_np / np.clip(alpha_np.sum(axis=1, keepdims=True), 1e-6, None)
            
            y_true_all.append(y_np)
            y_pred_all.append(pred_np)

    y_true_all = np.vstack(y_true_all)
    y_pred_all = np.vstack(y_pred_all)
    
    valid_mask = np.isfinite(y_true_all).all(axis=1) & np.isfinite(y_pred_all).all(axis=1)
    y_true_all = y_true_all[valid_mask]
    y_pred_all = y_pred_all[valid_mask]

    if y_true_all.shape[0] < 2:
        test_r2_mean = float("nan")
        test_r2_each = [float("nan")] * len(TARGET_FIELDS)
        test_rmse_each = [float("nan")] * len(TARGET_FIELDS)
        test_mae_each = [float("nan")] * len(TARGET_FIELDS)
    else:
        y_true_all = np.clip(y_true_all, 0.0, 1.0)
        y_pred_all = np.clip(y_pred_all, 0.0, 1.0)
        
        test_r2_each = [r2_score(y_true_all[:, i], y_pred_all[:, i]) 
                       for i in range(len(TARGET_FIELDS))]
        test_rmse_each = [np.sqrt(mean_squared_error(y_true_all[:, i], y_pred_all[:, i])) 
                         for i in range(len(TARGET_FIELDS))]
        test_mae_each = [mean_absolute_error(y_true_all[:, i], y_pred_all[:, i]) 
                        for i in range(len(TARGET_FIELDS))]
        test_r2_mean = float(np.mean(test_r2_each))

    metrics = {
        "fold": fold_id,
        "fraction": data_fraction,
        "model": model_kind,
        "val_loss": best_val,
        "test_r2_mean": test_r2_mean,
        "n_train": len(train_idx),
        "n_val": len(val_idx),
        "n_test": len(test_idx),
    }
    
    for i, name in enumerate(TARGET_FIELDS):
        metrics[f"test_r2_{name}"] = test_r2_each[i] if i < len(test_r2_each) else float("nan")
        metrics[f"test_rmse_{name}"] = test_rmse_each[i] if i < len(test_rmse_each) else float("nan")
        metrics[f"test_mae_{name}"] = test_mae_each[i] if i < len(test_mae_each) else float("nan")

    # 保存最佳模型（只保存全量）
    if data_fraction >= 0.99:
        model_path = os.path.join(OUT_DIR, f"{model_kind}_fold{fold_id}_best.pth")
        torch.save(best_state, model_path)
        print(f"✅ 模型已保存: {model_path}")

    return metrics


In [5]:
# =========================
# 5. 5折 × 多比例 总控(改进版)
# =========================
from sklearn.model_selection import KFold

def run_5fold_with_fractions():
    print("=" * 60)
    print("开始构建数据集...")
    print("=" * 60)
    
    full_ds = PatchPointDataset(
        LABEL_SHP,
        DAY_TIFS,
        NIGHT_TIF,
        TARGET_FIELDS,
        patch=PATCH_SIZE,
        augment=True
    )
    
    n_total = len(full_ds)
    all_idx = np.arange(n_total)

    model_list = []
    if RUN_DAY:
        model_list.append("day")
    if RUN_NIGHT:
        model_list.append("night")
    if RUN_FUSION:
        model_list.append("fusion")

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    all_results = []

    fold_id = 0
    for trainval_idx, test_idx in kf.split(all_idx):
        fold_id += 1

        np.random.seed(fold_id)
        np.random.shuffle(trainval_idx)
        n_val = max(1, int(len(trainval_idx) * 0.2))
        val_idx = trainval_idx[:n_val]
        train_idx_full = trainval_idx[n_val:]

        print(f"\n{'='*60}")
        print(f"Fold {fold_id}/5")
        print(f"{'='*60}")
        print(f"Train_full={len(train_idx_full)}, Val={len(val_idx)}, Test={len(test_idx)}")

        for mk in model_list:
            for frac in FRACTIONS:
                try:
                    print(f"\n--- Fold {fold_id}, Model={mk}, Fraction={frac*100:.0f}% ---")
                    m = train_one_fold_one_fraction(
                        full_ds,
                        train_idx_full=train_idx_full,
                        val_idx=val_idx,
                        test_idx=test_idx,
                        model_kind=mk,
                        data_fraction=frac,
                        fold_id=fold_id
                    )
                    all_results.append(m)
                    
                    # 实时保存
                    df_temp = pd.DataFrame(all_results)
                    temp_csv = os.path.join(OUT_DIR, "cv5_temp_results.csv")
                    df_temp.to_csv(temp_csv, index=False)
                    
                except Exception as e:
                    print(f"❌ Error in fold {fold_id}, model {mk}, frac {frac}: {e}")
                    import traceback
                    traceback.print_exc()
                    # 把错误也记录进去，防止最后是空表
                    all_results.append({
                        "fold": fold_id,
                        "fraction": frac,
                        "model": mk,
                        "error": str(e)
                    })
                    df_temp = pd.DataFrame(all_results)
                    temp_csv = os.path.join(OUT_DIR, "cv5_temp_results.csv")
                    df_temp.to_csv(temp_csv, index=False)
                    continue

    df = pd.DataFrame(all_results)
    csv_path = os.path.join(OUT_DIR, "cv5_all_fractions_results.csv")
    df.to_csv(csv_path, index=False)
    
    print(f"\n{'='*60}")
    print("✅ 5折×多比例 实验结果已保存:", csv_path)
    print(f"{'='*60}")
    
    print("\n📊 结果汇总:")
    for mk in model_list:
        for frac in FRACTIONS:
            sub = df[(df["model"] == mk) & (df["fraction"] == frac)]
            if not sub.empty and "test_r2_mean" in sub.columns:
                mean_r2 = sub["test_r2_mean"].mean()
                std_r2 = sub["test_r2_mean"].std()
                print(f"{mk} @ {frac*100:.0f}%: R² = {mean_r2:.3f} ± {std_r2:.3f}")
    
    full_ds.close()
    
    return df


In [6]:
# =========================
# 6. 全幅影像推理(改进版)
# =========================
import math
from tqdm import tqdm

def infer_full_raster(
    model_path,
    out_dir=OUT_DIR,
    patch_size=PATCH_SIZE,
    step=None,
    batch_size=16,
    tag=None,
):
    os.makedirs(out_dir, exist_ok=True)

    day_srcs = [rasterio.open(p) for p in DAY_TIFS]
    night_src = rasterio.open(NIGHT_TIF)

    base_meta = day_srcs[0].meta.copy()
    height = day_srcs[0].height
    width = day_srcs[0].width

    print(f"影像尺寸: {height} × {width}")

    model = FusionNet(
        day_channels=len(DAY_TIFS),
        night_channels=1,
        n_comp=len(TARGET_FIELDS)
    ).to(DEVICE)
    
    try:
        state_dict = torch.load(model_path, map_location=DEVICE)
        model.load_state_dict(state_dict)
        print(f"✅ 模型加载成功: {model_path}")
    except Exception as e:
        print(f"❌ 模型加载失败: {e}")
        return
    
    model.eval()

    out_paths = []
    out_files = []
    base_meta.update({
        "count": 1,
        "dtype": "float32",
        "compress": "lzw",
        "nodata": -9999
    })
    
    for name in TARGET_FIELDS:
        if tag:
            fname = f"pred_{tag}_{name}_2020_90m.tif"
        else:
            fname = f"pred_{name}_2020_90m.tif"
        op = os.path.join(out_dir, fname)
        out_paths.append(op)
        out_files.append(rasterio.open(op, "w", **base_meta))

    ps = patch_size
    if step is None:
        step = patch_size // 2

    windows = []
    for row in range(0, height, step):
        for col in range(0, width, step):
            win_h = min(ps, height - row)
            win_w = min(ps, width - col)
            windows.append((row, col, win_h, win_w))

    print(f"🗺️ 开始全幅推理 (共 {len(windows)} 个窗口)...")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(windows), batch_size), desc="推理进度"):
            batch_windows = windows[i:i+batch_size]
            batch_day = []
            batch_night = []
            valid_flags = []
            
            for row, col, win_h, win_w in batch_windows:
                window = rasterio.windows.Window(col, row, win_w, win_h)
                
                day_stack = []
                all_valid = True
                for src in day_srcs:
                    arr = src.read(1, window=window, boundless=True, masked=True)
                    if arr.mask.all():
                        all_valid = False
                        break
                    day_stack.append(arr.filled(0).astype(np.float32))
                
                night_arr = night_src.read(1, window=window, boundless=True, masked=True)
                if night_arr.mask.all():
                    all_valid = False
                
                valid_flags.append(all_valid)
                
                if not all_valid:
                    batch_day.append(None)
                    batch_night.append(None)
                    continue
                
                day_arr = np.stack(day_stack, axis=0)
                night_arr = np.expand_dims(night_arr.filled(0).astype(np.float32), axis=0)
                
                if win_h != ps or win_w != ps:
                    pad_day = np.zeros((day_arr.shape[0], ps, ps), dtype=np.float32)
                    pad_night = np.zeros((1, ps, ps), dtype=np.float32)
                    pad_day[:, :win_h, :win_w] = day_arr
                    pad_night[:, :win_h, :win_w] = night_arr
                    day_arr = pad_day
                    night_arr = pad_night
                
                batch_day.append(day_arr)
                batch_night.append(night_arr)
            
            for idx, (all_valid, (row, col, win_h, win_w)) in enumerate(zip(valid_flags, batch_windows)):
                window = rasterio.windows.Window(col, row, win_w, win_h)
                
                if not all_valid:
                    for o in out_files:
                        o.write(np.full((win_h, win_w), -9999, dtype=np.float32), 1, window=window)
                    continue
                
                day_t = torch.from_numpy(batch_day[idx]).unsqueeze(0).to(DEVICE)
                night_t = torch.from_numpy(batch_night[idx]).unsqueeze(0).to(DEVICE)
                
                alpha = model(day_t, night_t)
                alpha = alpha.squeeze(0).cpu().numpy()
                
                denom = np.clip(alpha.sum(axis=0, keepdims=True), 1e-6, None)
                comp = alpha / denom
                
                for comp_idx, o in enumerate(out_files):
                    o.write(comp[comp_idx, :win_h, :win_w].astype(np.float32), 1, window=window)

    for src in day_srcs:
        src.close()
    night_src.close()
    for o in out_files:
        o.close()

    print("✅ 推理完成,输出到:")
    for p in out_paths:
        print(f"  - {p}")


In [7]:
# =========================
# 7. 可视化评估结果(改进版，完整版)
# =========================
def plot_from_cv_results(
    csv_path=os.path.join(OUT_DIR, "cv5_all_fractions_results.csv"),
    out_dir=OUT_DIR
):
    """
    从交叉验证结果csv生成多种图：
    1) 各模型：训练比例 vs 平均R² 折线图
    2) 单模型(100%数据)：四个组成变量的R²箱线图
    3) 多模型：对比折线图
    4) 单模型(100%数据)：各折×各变量的R²热力图
    """
    import pandas as pd
    import matplotlib.pyplot as plt

    # 可选的 seaborn，美化图形；没有也能跑
    try:
        import seaborn as sns
        sns.set_palette("husl")
        use_sns = True
    except ImportError:
        use_sns = False

    if not os.path.exists(csv_path):
        print(f"⚠️ 找不到结果文件: {csv_path}")
        return

    df = pd.read_csv(csv_path)
    if df.empty or "model" not in df.columns:
        print("⚠️ 结果表为空，或者缺少 'model' 列，无法绘图。")
        return

    os.makedirs(out_dir, exist_ok=True)

    models = df["model"].dropna().unique()

    # ========= 图1: 数据比例 vs R² =========
    fig, axes = plt.subplots(1, len(models), figsize=(6 * len(models), 5))
    if len(models) == 1:
        axes = [axes]

    for idx, mk in enumerate(models):
        sub = df[(df["model"] == mk) & (df["fraction"].notna())]

        # 只对有 test_r2_mean 的做聚合
        sub = sub[sub["test_r2_mean"].notna()]

        g = sub.groupby("fraction")["test_r2_mean"].agg(["mean", "std", "count"]).reset_index()

        ax = axes[idx]
        ax.errorbar(
            g["fraction"] * 100,
            g["mean"],
            yerr=g["std"].fillna(0),
            fmt="-o",
            linewidth=2.2,
            markersize=7,
            capsize=5,
            capthick=1.5,
        )
        ax.set_xlabel("Percent of training data used (%)", fontsize=12, fontweight="bold")
        ax.set_ylabel("Held-out mean R²", fontsize=12, fontweight="bold")
        ax.set_title(f"Performance vs Data Fraction ({mk.upper()})", fontsize=13, fontweight="bold")
        ax.grid(True, alpha=0.3)

        # 适当设置 y 范围
        y_min = (g["mean"] - g["std"].fillna(0)).min()
        y_max = (g["mean"] + g["std"].fillna(0)).max()
        ax.set_ylim([max(-0.1, y_min - 0.05), min(1.05, y_max + 0.05)])

    plt.tight_layout()
    out_png = os.path.join(out_dir, "fig_fraction_vs_r2_all_models.png")
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"📈 已输出: {out_png}")

    # ========= 图2: 每个模型在100%数据下，各变量的R²箱线图 =========
    # 我们只画 fraction >= 0.999 的，也就是全量数据
    for mk in models:
        sub = df[(df["model"] == mk) & (df["fraction"] >= 0.999)]
        if sub.empty:
            continue

        # 找出所有 test_r2_ 前缀的列
        r2_cols = [c for c in sub.columns if c.startswith("test_r2_") and c != "test_r2_mean"]
        if not r2_cols:
            continue

        data = []
        labels = []
        for col in r2_cols:
            var_name = col.replace("test_r2_", "")
            data.append(sub[col].values)
            labels.append(var_name)

        fig, ax = plt.subplots(figsize=(10, 6))
        bp = ax.boxplot(
            data,
            labels=labels,
            patch_artist=True,
            notch=True,
            showmeans=True,
        )

        if use_sns:
            palette = sns.color_palette("husl", len(labels))
        else:
            # 兜底简单颜色
            palette = [(0.6, 0.8, 1.0)] * len(labels)

        for patch, color in zip(bp["boxes"], palette):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)

        ax.set_ylabel("R² Score", fontsize=12, fontweight="bold")
        ax.set_xlabel("Land Cover Component", fontsize=12, fontweight="bold")
        ax.set_title(
            f"Per-Variable R² Distribution ({mk.upper()}, 100% data)",
            fontsize=13,
            fontweight="bold",
        )
        ax.grid(True, axis="y", alpha=0.3)

        plt.tight_layout()
        out_png = os.path.join(out_dir, f"fig_per_variable_box_{mk}.png")
        plt.savefig(out_png, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"📦 已输出: {out_png}")

    # ========= 图3: 多模型对比（如果你一次跑了 day/night/fusion 就能看出来） =========
    if len(models) > 1:
        fig, ax = plt.subplots(figsize=(11, 6))
        for mk in models:
            sub = df[(df["model"] == mk) & (df["fraction"].notna())]
            sub = sub[sub["test_r2_mean"].notna()]
            g = sub.groupby("fraction")["test_r2_mean"].agg(["mean", "std"]).reset_index()
            ax.errorbar(
                g["fraction"] * 100,
                g["mean"],
                yerr=g["std"].fillna(0),
                fmt="-o",
                linewidth=2.2,
                markersize=7,
                capsize=4,
                label=mk.upper(),
            )

        ax.set_xlabel("Percent of training data used (%)", fontsize=12, fontweight="bold")
        ax.set_ylabel("Held-out mean R²", fontsize=12, fontweight="bold")
        ax.set_title("Model Comparison", fontsize=14, fontweight="bold")
        ax.legend(fontsize=11, frameon=True, shadow=True)
        ax.grid(True, alpha=0.3)

        plt.tight_layout()
        out_png = os.path.join(out_dir, "fig_model_comparison.png")
        plt.savefig(out_png, dpi=300, bbox_inches="tight")
        plt.close()
        print(f"🔄 已输出: {out_png}")

    # ========= 图4: 热力图 - 各变量在不同折的表现(只看100%数据) =========
    # 这里用 seaborn 会更好看，没有 seaborn 就用 matplotlib 简单画
    for mk in models:
        sub = df[(df["model"] == mk) & (df["fraction"] >= 0.999)]
        if sub.empty:
            continue

        r2_cols = [c for c in sub.columns if c.startswith("test_r2_") and c != "test_r2_mean"]
        if not r2_cols:
            continue

        # 构造成 (变量 × 折) 的表
        heatmap_data = sub[["fold"] + r2_cols].set_index("fold")
        heatmap_data.columns = [c.replace("test_r2_", "") for c in heatmap_data.columns]

        if use_sns:
            import seaborn as sns
            fig, ax = plt.subplots(figsize=(10, 6))
            sns.heatmap(
                heatmap_data.T,
                annot=True,
                fmt=".3f",
                cmap="YlGnBu",
                cbar_kws={"label": "R² Score"},
                ax=ax,
                vmin=0.0,
                vmax=1.0,
            )
            ax.set_xlabel("Fold", fontsize=12, fontweight="bold")
            ax.set_ylabel("Variable", fontsize=12, fontweight="bold")
            ax.set_title(
                f"R² Heatmap Across Folds ({mk.upper()}, 100% data)",
                fontsize=13,
                fontweight="bold",
            )
            plt.tight_layout()
            out_png = os.path.join(out_dir, f"fig_r2_heatmap_{mk}.png")
            plt.savefig(out_png, dpi=300, bbox_inches="tight")
            plt.close()
            print(f"🔥 已输出: {out_png}")
        else:
            # 没有 seaborn 的简易版
            fig, ax = plt.subplots(figsize=(10, 6))
            im = ax.imshow(heatmap_data.T, aspect="auto", vmin=0.0, vmax=1.0, cmap="YlGnBu")
            ax.set_xticks(range(len(heatmap_data.index)))
            ax.set_xticklabels(heatmap_data.index)
            ax.set_yticks(range(len(heatmap_data.columns)))
            ax.set_yticklabels(heatmap_data.columns)
            ax.set_xlabel("Fold")
            ax.set_ylabel("Variable")
            ax.set_title(f"R² Heatmap Across Folds ({mk.upper()}, 100% data)")
            fig.colorbar(im, ax=ax, label="R² Score")
            plt.tight_layout()
            out_png = os.path.join(out_dir, f"fig_r2_heatmap_{mk}.png")
            plt.savefig(out_png, dpi=300, bbox_inches="tight")
            plt.close()
            print(f"🔥 已输出(简易版): {out_png}")

    print("\n✅ 所有图表生成完成!")


In [ ]:
# =========================
# 8. 主函数
# =========================
if __name__ == "__main__":
    print("="*70)
    print(" Landsat + 夜光数据深度学习组成预测系统")
    print("="*70)
    
    # 步骤1: 运行5折交叉验证
    print("\n📊 步骤1: 运行5折交叉验证实验...")
    try:
        df_results = run_5fold_with_fractions()
    except Exception as e:
        print(f"❌ 交叉验证失败: {e}")
        import traceback
        traceback.print_exc()
        df_results = None
    
    # 步骤2: 生成评估图表
    if df_results is not None:
        print("\n📈 步骤2: 生成评估图表...")
        try:
            plot_from_cv_results()
        except Exception as e:
            print(f"❌ 图表生成失败: {e}")
            import traceback
            traceback.print_exc()
    
    # 步骤3: 自动选择 R² 最高的融合模型进行全幅影像推理
    print("\n🗺️ 步骤3: 根据最佳 R² 自动选择模型并制图...")
    try:
        import pandas as pd

        csv_path = os.path.join(OUT_DIR, "cv5_all_fractions_results.csv")
        df = pd.read_csv(csv_path)

        df_fusion = df[(df.get("model") == "fusion") & (df.get("fraction", 0) >= 0.999)]
        if df_fusion.empty:
            raise RuntimeError("没有找到 fusion 模型的全量数据结果，无法自动选择最佳 fold。")

        best_row = df_fusion.sort_values("test_r2_mean", ascending=False).iloc[0]
        best_fold = int(best_row["fold"])
        best_r2 = float(best_row["test_r2_mean"])
        print(f"🌟 最佳模型: fusion_fold{best_fold}, R²={best_r2:.3f}")

        best_model_path = os.path.join(OUT_DIR, f"fusion_fold{best_fold}_best.pth")
        if os.path.exists(best_model_path):
            infer_full_raster(
                model_path=best_model_path,
                out_dir=os.path.join(OUT_DIR, "maps_2020"),
                step=32,
                tag=f"fusion_fold{best_fold}"
            )
        else:
            print(f"⚠️ 未找到对应的模型文件: {best_model_path}")

    except Exception as e:
        print(f"❌ 自动选择模型推理失败: {e}")
        import traceback
        traceback.print_exc()
    
    print("\n" + "="*70)
    print(" 🎉 所有任务完成!")
    print("="*70)


 Landsat + 夜光数据深度学习组成预测系统

📊 步骤1: 运行5折交叉验证实验...
开始构建数据集...
⏳ 正在检查哪些样本在影像上有值...
✅ 可用样本数: 38499 / 38577

Fold 1/5
Train_full=24640, Val=6159, Test=7700

--- Fold 1, Model=fusion, Fraction=5% ---
[Fold 1][frac 0.05] epoch 1/100 train_loss=-1.5385 val_loss=-0.1998 meanR²=-0.250
